# Forecasting the sale of stores using ML models

## 1 - Importing Libraries

In [9]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import keras as ks
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
import random as rd

print("numpy version: ", np.__version__)
print("pandas version: ", pd.__version__)
print("keras version: ", ks.__version__)


numpy version:  1.23.5
pandas version:  2.0.0
keras version:  2.9.0


# 2 - Importing the training data

In [3]:
list_of_datasets = []

for i in range(1,55):
    list_of_datasets.append(pd.read_csv("data_store_sales_preprocessed/store_"+str(i)+'_train.csv'))

# Quick look at the data
list_of_datasets[0].head()

,date,store_number,store_type,store_cluster,store_city,store_state,day_of_the_week,is_a_special_day,type,locale,...,POULTRY_sales,POULTRY_onpromotion,PREPARED FOODS_sales,PREPARED FOODS_onpromotion,PRODUCE_sales,PRODUCE_onpromotion,SCHOOL AND OFFICE SUPPLIES_sales,SCHOOL AND OFFICE SUPPLIES_onpromotion,SEAFOOD_sales,SEAFOOD_onpromotion
0,2013-01-01,1,0,13,0,0,1,1,Holiday,National,...,0.00000,0,0.0,0,0.0,0,0.0,0,0.000000,0
1,2013-01-02,1,0,13,0,0,2,0,NONE,NONE,...,247.29700,0,47.0,0,0.0,0,0.0,0,38.029000,0
2,2013-01-03,1,0,13,0,0,3,0,NONE,NONE,...,187.27800,0,63.0,0,0.0,0,0.0,0,17.366001,0
3,2013-01-04,1,0,13,0,0,4,0,NONE,NONE,...,258.02300,0,67.0,0,0.0,0,0.0,0,29.907001,0
4,2013-01-05,1,0,13,0,0,5,1,Work Day,National,...,212.33301,0,66.0,0,0.0,0,0.0,0,24.842000,0


Let's transform classification data into numeric data

In [6]:
for i in range(0,54):
    list_of_datasets[i]['type'] = pd.factorize(list_of_datasets[i]['type'])[0]
    list_of_datasets[i]['locale'] = pd.factorize(list_of_datasets[i]['locale'])[0]
    list_of_datasets[i]['locale_name'] = pd.factorize(list_of_datasets[i]['locale_name'])[0]
    list_of_datasets[i]['description'] = pd.factorize(list_of_datasets[i]['description'])[0]
    list_of_datasets[i]['transferred'] = pd.factorize(list_of_datasets[i]['transferred'])[0]

# Quick look at the data
list_of_datasets[0].head()


,date,store_number,store_type,store_cluster,store_city,store_state,day_of_the_week,is_a_special_day,type,locale,...,POULTRY_sales,POULTRY_onpromotion,PREPARED FOODS_sales,PREPARED FOODS_onpromotion,PRODUCE_sales,PRODUCE_onpromotion,SCHOOL AND OFFICE SUPPLIES_sales,SCHOOL AND OFFICE SUPPLIES_onpromotion,SEAFOOD_sales,SEAFOOD_onpromotion
0,2013-01-01,1,0,13,0,0,1,1,0,0,...,0.00000,0,0.0,0,0.0,0,0.0,0,0.000000,0
1,2013-01-02,1,0,13,0,0,2,0,1,1,...,247.29700,0,47.0,0,0.0,0,0.0,0,38.029000,0
2,2013-01-03,1,0,13,0,0,3,0,1,1,...,187.27800,0,63.0,0,0.0,0,0.0,0,17.366001,0
3,2013-01-04,1,0,13,0,0,4,0,1,1,...,258.02300,0,67.0,0,0.0,0,0.0,0,29.907001,0
4,2013-01-05,1,0,13,0,0,5,1,2,0,...,212.33301,0,66.0,0,0.0,0,0.0,0,24.842000,0


Getting the training the inputs, the input is going whill contained 100 days with all the features. The output will be the 15 next day's sales for the store (15 x 33)

In [ ]:
number_of_days = len(list_of_datasets[0])
ratio = 0.8 # 80% of the data is used for training



list_of_stores_training_dates = []
list_of_stores_validation_dates = []

possibilities = []
for i in range(0,number_of_days-115):
    possibilities.append([i,i+115])



for i in range(0,54):
    rd.shuffle(possibilities)
    list_of_stores_training_dates.append(possibilities[0:int(ratio*len(possibilities))])
    list_of_stores_validation_dates.append(possibilities[int(ratio*len(possibilities)):len(possibilities)])
